In [2]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import urllib.request
import camelot

In [27]:
with open('../Data/Raw/116_congress_twitter.html') as fp:
    soup = BeautifulSoup(fp, 'html.parser')

table = soup.find('table', attrs={'id':"footable_16836"})

l = []
for tr in table.findAll('tr'):
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
Data116 = pd.DataFrame(l[1:], columns = [header.getText() for header in table.findAll('th')]).drop(columns = ['Name Links', 'Twitter Links', 'Instagram', 'Facebook Page', 'Facebook'])


In [28]:
rename_chamber = {'U.S. Representative': 'Representative', 'U.S. Senator': 'Senator'}
Data116 = Data116.replace(rename_chamber).rename(columns = {'Chamber of Congress': 'Type'})

In [36]:
us_state_abbrev = {
    r'Alabama.*': 'AL',
    r'Alaska.*': 'AK',
    r'American Samoa.*': 'AS',
    r'Arizona.*': 'AZ',
    r'Arkansas.*': 'AR',
    r'California.*': 'CA',
    r'Colorado.*': 'CO',
    r'Connecticut.*': 'CT',
    r'Delaware.*': 'DE',
    r'District of Columbia.*': 'DC',
    r'Florida.*': 'FL',
    r'Georgia.*': 'GA',
    r'Guam.*': 'GU',
    r'Hawaii.*': 'HI',
    r'Idaho.*': 'ID',
    r'Illinois.*': 'IL',
    r'Indiana.*': 'IN',
    r'Iowa.*': 'IA',
    r'Kansas.*': 'KS',
    r'Kentucky.*': 'KY',
    r'Louisiana.*': 'LA',
    r'Maine.*': 'ME',
    r'Maryland.*': 'MD',
    r'Massachusetts.*': 'MA',
    r'Michigan.*': 'MI',
    r'Minnesota.*': 'MN',
    r'Mississippi.*': 'MS',
    r'Missouri.*': 'MO',
    r'Montana.*': 'MT',
    r'Nebraska.*': 'NE',
    r'Nevada.*': 'NV',
    r'New Hampshire.*': 'NH',
    r'New Jersey.*': 'NJ',
    r'New Mexico.*': 'NM',
    r'New York.*': 'NY',
    r'North Carolina.*': 'NC',
    r'North Dakota.*': 'ND',
    r'Northern Mariana Islands.*':'MP',
    r'Ohio.*': 'OH',
    r'Oklahoma.*': 'OK',
    r'Oregon.*': 'OR',
    r'Pennsylvania.*': 'PA',
    r'Puerto Rico.*': 'PR',
    r'Rhode Island.*': 'RI',
    r'South Carolina.*': 'SC',
    r'South Dakota.*': 'SD',
    r'Tennessee.*': 'TN',
    r'Texas.*': 'TX',
    r'Utah.*': 'UT',
    r'Vermont.*': 'VT',
    r'Virgin Islands.*': 'VI',
    r'Virginia.*': 'VA',
    r'Washington.*': 'WA',
    r'West V.*': 'WV', # Written in different ways
    r'Wisconsin.*': 'WI',
    r'Wyoming.*': 'WY'
}

Data116 = Data116.replace(regex = us_state_abbrev)
Data116 = Data116.replace(regex = {r'^@': ''})
Data116 = Data116[Data116.Name != "Vacant"]
Data116

,State,Type,Name,Party,Twitter
0,AL,Senator,Richard Shelby,R,SenShelby
1,AL,Senator,Doug Jones,D,DougJones
2,AL,Representative,"Byrne, Bradley",R,RepByrne
3,AL,Representative,"Roby, Martha",R,RepMarthaRoby
4,AL,Representative,"Rogers, Mike",R,RepMikeRogersAL
...,...,...,...,...,...
536,WI,Representative,"Tiffany, Thomas",R,TomTiffanyWI
537,WI,Representative,"Gallagher, Mike",R,MikeforWI
538,WY,Senator,"Enzi, Mike",R,SenatorEnzi
539,WY,Senator,"Barrasso, John",R,SenJohnBarrasso


In [37]:
file115 = '../Data/Raw/115_congress_twitter.pdf'
tables = camelot.read_pdf(file115, pages = 'all')

Data115 = pd.DataFrame(np.concatenate([d.df.drop(0).values for d in tables]), columns=tables[0].df.iloc[0]).drop(columns = "District")
rename_chamber = {'Rep.': 'Representative', 'Sen.': 'Senator'}
Data115 = Data115.replace(rename_chamber)


Data115["Name"] = Data115["First Name"] + " " + Data115["Last Name"]
Data115 = Data115.drop(columns = ["First Name", "Last Name"])

Data115 = Data115.rename(columns = {'Title': 'Type', "Twitter Handle": "Twitter"})

In [38]:
Data_Full = Data115.append(Data116, ignore_index = True)
Data_Full

,State,Party,Type,Twitter,Name
0,AK,R,Representative,repdonyoung,Don Young
1,AK,R,Senator,lisamurkowski,Lisa Murkowski
2,AK,R,Senator,sendansullivan,Dan Sullivan
3,AL,R,Representative,Robert_Aderholt,Robert Aderholt
4,AL,R,Representative,RepMoBrooks,Mo Brooks
...,...,...,...,...,...
1067,WI,R,Representative,TomTiffanyWI,"Tiffany, Thomas"
1068,WI,R,Representative,MikeforWI,"Gallagher, Mike"
1069,WY,R,Senator,SenatorEnzi,"Enzi, Mike"
1070,WY,R,Senator,SenJohnBarrasso,"Barrasso, John"


In [13]:
Data_Full[Data_Full.duplicated(subset = ["Twitter"])]

,State,Party,Type,Twitter,Name
535,AL,R,Senator,SenShelby,Richard Shelby
537,AL,R,Representative,RepByrne,"Byrne, Bradley"
538,AL,R,Representative,RepMarthaRoby,"Roby, Martha"
539,AL,R,Representative,RepMikeRogersAL,"Rogers, Mike"
540,AL,R,Representative,Robert_Aderholt,"Aderholt, Robert"
...,...,...,...,...,...
1069,WI,R,Representative,JimPressOffice,"Sensenbrenner, F. James"
1070,WI,R,Representative,RepGrothman,"Grothman, Glenn"
1072,WI,R,Representative,MikeforWI,"Gallagher, Mike"
1073,WY,R,Senator,SenatorEnzi,"Enzi, Mike"


In [43]:
Data_Full[(Data_Full.duplicated(subset = ["Twitter"])) & ~(Data_Full.duplicated(subset = ["State", "Twitter"]))]

,State,Party,Type,Twitter,Name
547,AS,R,Representative,RepAmata,"Radewagen, Amata"
648,FL,R,Representative,congbillposey,"Posey, Bill"
699,IL,D,Representative,RepMikeQuigley,"Quigley, Mike"
889,NC,D,Representative,RepDavidEPrice,"Price, David"
1021,UT,R,Representative,RepHalRogers,"Bishop, Rob"


In [45]:
Data_Full[(Data_Full.duplicated(subset = ["Twitter"], keep = 'last')) & ~(Data_Full.duplicated(subset = ["State", "Twitter"], keep = 'last'))]

,State,Party,Type,Twitter,Name
176,IL,D,Representative,RepDavidEPrice,Mike Quigley
233,MD,D,Representative,RepMikeQuigley,Jamie Raskin
297,NC,D,Representative,congbillposey,David Price
472,TX,R,Representative,RepAmata,John Ratcliffe
742,KY,R,Representative,RepHalRogers,"Rogers, Harold"


In [41]:
Data_Full[Data_Full.State == "PR"]

,State,Party,Type,Twitter,Name
416,PR,R,Representative,RepJenniffer,Jenniffer Gonzalez-Colon
934,PR,R,Representative,Jenniffer2012,"Gonzalez-Colon, Jenniffer"


In [42]:
Data116.State.unique()

array(['AL', 'AK', 'AS', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'DC', 'FL',
       'GA', 'GU', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME',
       'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ',
       'NM', 'NY', 'NC', 'ND', 'MP', 'OH', 'OK', 'OR', 'PR', 'PA', 'RI',
       'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'VI', 'WA', 'WV', 'WI',
       'WY'], dtype=object)